In [ ]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 KB 14.0 MB/s eta 0:00:00


In [ ]:

import pandas as pd
import numpy as np
import random

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

from google.colab import drive 
from google.colab import files

import json

import requests
import time
from pprint import pprint

import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

# TensorFlow and tf.keras
import tensorflow.compat.v1 as tf

from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt


import os

KeyboardInterrupt: ignored

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
f = open('/content/drive/MyDrive/Spotify Playlist Enhancer/Deep Learning/mpd.slice.0-999.json')
playlists = json.load(f)['playlists']

In [ ]:
song_df = pd.read_csv('/content/drive/MyDrive/Spotify Playlist Enhancer/Deep Learning/tracks_features.csv')
#song_df =  song_df[song_df['year'] > 1500]
song_df.head()
#the above is to get rid of any outliers, mostly the jimmy neutron

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


In [ ]:
CID = "aca1ad95b3a945dcb8f3dc98f9ac87bc"
CSECRET = "9dc90362f053404d8c85a15784ce9c8a"
RURI = "http://localhost/"
SCOPE = "user-library-read"

client_credentials_manager = SpotifyClientCredentials(client_id=CID, client_secret=CSECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

Trying to use a modified sigmoid function to normalize our data



In [3]:
def sigmoid(x):
  return (2 / (1 + np.exp(-x))) - 1

def output_to_json(output_name, playlist_output):
  json_output = json.dumps(playlist_output)
  with open(f"output_{output_name}.json", "w") as outfile:
    outfile.write(json_output)
  files.download(f'output_{output_name}.json')

def import_json(input_bounds):
  f = open(f'/content/drive/MyDrive/Spotify Playlist Enhancer/Deep Learning/mpd.slice.{input_bounds}.json')
  return json.load(f)['playlists']

In [ ]:
output_to_json('1009-1999', playlists)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
song_list = ['7lmeHLHBe4nmXzuXc0HDjk','1wsRitfRRtWyEapl0q22o8']
sp.audio_features(song_list)

[{'danceability': 0.47,
  'energy': 0.978,
  'key': 7,
  'loudness': -5.399,
  'mode': 1,
  'speechiness': 0.0727,
  'acousticness': 0.0261,
  'instrumentalness': 1.09e-05,
  'liveness': 0.356,
  'valence': 0.503,
  'tempo': 117.906,
  'type': 'audio_features',
  'id': '7lmeHLHBe4nmXzuXc0HDjk',
  'uri': 'spotify:track:7lmeHLHBe4nmXzuXc0HDjk',
  'track_href': 'https://api.spotify.com/v1/tracks/7lmeHLHBe4nmXzuXc0HDjk',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7lmeHLHBe4nmXzuXc0HDjk',
  'duration_ms': 210133,
  'time_signature': 4},
 {'danceability': 0.599,
  'energy': 0.957,
  'key': 11,
  'loudness': -5.764,
  'mode': 1,
  'speechiness': 0.188,
  'acousticness': 0.0129,
  'instrumentalness': 7.06e-05,
  'liveness': 0.155,
  'valence': 0.489,
  'tempo': 103.68,
  'type': 'audio_features',
  'id': '1wsRitfRRtWyEapl0q22o8',
  'uri': 'spotify:track:1wsRitfRRtWyEapl0q22o8',
  'track_href': 'https://api.spotify.com/v1/tracks/1wsRitfRRtWyEapl0q22o8',
  'analysis_url': 'http

The below was my first implementation, it sorted through our million song list and tried to find what it could. it was often unsusseccsful

The following one uses the API and is much better(it can find all songs)

In [ ]:
def convert_playlists_by_batch(playlists, n = 20):
  iteration = 0
  modified_playlists = []
  for playlist in playlists:
    new_playlist = { 'tracks': [], 'hidden_song': []}
    if playlist['num_tracks'] > n:
      current_playlist = playlist['num_tracks']
      selected_tracks = random.sample(playlist['tracks'], k = n + 1)
      selected_tracks_uris = [song['track_uri'] for song in selected_tracks]
      selected_tracks_features = [list(sp.audio_features(song_id)[0].values())[0:11] for  song_id in selected_tracks_uris]
      for i in range(n + 1):
        selected_tracks_features[i][10] = selected_tracks_features[i][10] - 95
        selected_tracks_features[i] = list(map(sigmoid, selected_tracks_features[i]))
      i = random.randrange(n + 1)
      new_playlist['hidden_song'] = selected_tracks_features.pop(i)
      new_playlist['tracks'] = selected_tracks_features
      print(f'Playlist {iteration}')
      modified_playlists = modified_playlists + [new_playlist]
    iteration += 1
  return modified_playlists

In [ ]:
def convert_playlists(playlists, n = 20):
  iteration = 0
  modified_playlists = []
  for playlist in playlists:
    new_playlist = { 'tracks': [], 'hidden_song': []}
    if playlist['num_tracks'] > n:
      current_playlist = playlist['num_tracks']
      selected_tracks = random.sample(playlist['tracks'], k = n + 1)
      
      for song in selected_tracks:
        song_id = song['track_uri'].split(':')[2]
        song_characteristics = list(sp.audio_features(song_id)[0].values())[0:11]
      
        #the first 10 song features. 10th is tempo, this might not be needed
        song_characteristics[10] = song_characteristics[10] - 95
        #subtracting tempo by the average song tempo so that more information shows up after scaling
        song_characteristics = list(map(sigmoid, song_characteristics))
        #normalizing the data
        new_playlist['tracks'] = new_playlist['tracks'] + [song_characteristics]

      i = random.randrange(n + 1)
      new_playlist['hidden_song'] = new_playlist['tracks'].pop(i)
      print(f'Playlist {iteration}')
      modified_playlists = modified_playlists + [new_playlist]
    iteration += 1
  return modified_playlists


The following code converts all the playlists on our drive and downloads them

In [ ]:
list_of_outputs = [[] for i in range(100)]

In [ ]:

for i in range(2,10):
  input_range = f'{1000*i}-{1000*i + 999}'
  playlists = import_json(input_range)
  print(f'Imported {len(playlists)} playlists in the range {input_range}')
  converted_playlists = convert_playlists(playlists)
  print(f'Done converting the range{input_range}')
  list_of_outputs[i] = converted_playlists
  output_to_json(input_range, list_of_outputs[i])

Imported 1000 playlists in the range 2000-2999
Playlist 0
Playlist 2
Playlist 3
Playlist 5
Playlist 6
Playlist 8
Playlist 10
Playlist 11
Playlist 12
Playlist 13
Playlist 14
Playlist 17
Playlist 18
Playlist 19
Playlist 20
Playlist 21
Playlist 24
Playlist 25
Playlist 26
Playlist 27
Playlist 31
Playlist 33
Playlist 34
Playlist 35
Playlist 36
Playlist 37
Playlist 38
Playlist 39
Playlist 40
Playlist 41
Playlist 42
Playlist 43
Playlist 44
Playlist 45
Playlist 46
Playlist 47
Playlist 49
Playlist 50
Playlist 51
Playlist 52
Playlist 53
Playlist 54
Playlist 55
Playlist 57
Playlist 60
Playlist 61
Playlist 62
Playlist 63
Playlist 64
Playlist 66
Playlist 67
Playlist 68
Playlist 69
Playlist 70
Playlist 71
Playlist 72
Playlist 73
Playlist 74
Playlist 75
Playlist 76
Playlist 77
Playlist 78
Playlist 79
Playlist 80
Playlist 81
Playlist 82
Playlist 83
Playlist 84
Playlist 86
Playlist 87
Playlist 88
Playlist 89
Playlist 90
Playlist 91
Playlist 92
Playlist 93
Playlist 94
Playlist 95
Playlist 96
Playlist 97

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Imported 1000 playlists in the range 3000-3999
Playlist 1
Playlist 3
Playlist 4
Playlist 5
Playlist 6
Playlist 7
Playlist 10
Playlist 11
Playlist 12
Playlist 13
Playlist 14
Playlist 15
Playlist 16
Playlist 17
Playlist 18
Playlist 20
Playlist 21
Playlist 22
Playlist 23
Playlist 24
Playlist 25
Playlist 26
Playlist 27
Playlist 28
Playlist 29
Playlist 30
Playlist 31
Playlist 32
Playlist 34
Playlist 35
Playlist 36
Playlist 37
Playlist 38
Playlist 40
Playlist 41
Playlist 42
Playlist 44
Playlist 46
Playlist 48
Playlist 49
Playlist 50
Playlist 51
Playlist 52
Playlist 53
Playlist 54
Playlist 55
Playlist 56
Playlist 57
Playlist 58
Playlist 59
Playlist 60
Playlist 61
Playlist 62
Playlist 63
Playlist 64
Playlist 65
Playlist 66
Playlist 67
Playlist 68
Playlist 69
Playlist 71
Playlist 72
Playlist 73
Playlist 75
Playlist 76
Playlist 77
Playlist 78
Playlist 80
Playlist 81
Playlist 82
Playlist 83
Playlist 84
Playlist 85
Playlist 86
Playlist 88
Playlist 89
Playlist 92
Playlist 93
Playlist 94
Playlist 95

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Imported 1000 playlists in the range 4000-4999
Playlist 0
Playlist 3
Playlist 4
Playlist 7
Playlist 8
Playlist 9
Playlist 10
Playlist 11
Playlist 12
Playlist 13
Playlist 14
Playlist 15
Playlist 17
Playlist 18
Playlist 19
Playlist 20
Playlist 21
Playlist 22
Playlist 23
Playlist 24
Playlist 25
Playlist 26
Playlist 27
Playlist 28
Playlist 29
Playlist 31
Playlist 32
Playlist 33
Playlist 34
Playlist 35
Playlist 36
Playlist 38
Playlist 42
Playlist 43
Playlist 45
Playlist 46
Playlist 47
Playlist 49
Playlist 50
Playlist 51
Playlist 52
Playlist 54
Playlist 55
Playlist 56
Playlist 57
Playlist 58
Playlist 59
Playlist 60
Playlist 61
Playlist 62
Playlist 63
Playlist 66
Playlist 67
Playlist 68
Playlist 70
Playlist 71
Playlist 72
Playlist 73
Playlist 74
Playlist 75
Playlist 76
Playlist 77
Playlist 78
Playlist 80
Playlist 81
Playlist 83
Playlist 85
Playlist 87
Playlist 88
Playlist 89
Playlist 91
Playlist 92
Playlist 93
Playlist 94
Playlist 95
Playlist 96
Playlist 97
Playlist 99
Playlist 101
Playlist 1

ConnectionError: ignored

In [ ]:
list_of_outputs

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
len(modified_playlists)

817

In [ ]:
features_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'year']
song_features = song_df[features_list]
for column in song_features.columns:
  song_features[column] = (song_features[column] - song_features[column].min()) / (song_features[column].max() - song_features[column].min())
song_features.head()
print(len(song_features))
len(song_df)

1204025


<ipython-input-12-a5943b93359e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_features[column] = (song_features[column] - song_features[column].min()) / (song_features[column].max() - song_features[column].min())


1204025

In [ ]:
json_output = json.dumps(modified_playlists)

In [ ]:
json_output

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
with open("json_output_1.json", "w") as outfile:
    outfile.write(json_output)
files.download('json_output_1.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
output_to_json(2, output2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
output2[0]

{'tracks': [[0.20983620392641567,
   0.4337447262800538,
   0.7615941559557646,
   -0.9976113999390553,
   0.4621171572600098,
   0.035035654045366904,
   0.0008099998228530136,
   0.0524518187441374,
   0.0394295473542321,
   0.26315952320398317,
   0.999999999999976],
  [0.20983620392641567,
   0.4337447262800538,
   0.7615941559557646,
   -0.9976113999390553,
   0.4621171572600098,
   0.035035654045366904,
   0.0008099998228530136,
   0.0524518187441374,
   0.0394295473542321,
   0.26315952320398317,
   0.999999999999976],
  [0.20983620392641567,
   0.4337447262800538,
   0.7615941559557646,
   -0.9976113999390553,
   0.4621171572600098,
   0.035035654045366904,
   0.0008099998228530136,
   0.0524518187441374,
   0.0394295473542321,
   0.26315952320398317,
   0.999999999999976],
  [0.20983620392641567,
   0.4337447262800538,
   0.7615941559557646,
   -0.9976113999390553,
   0.4621171572600098,
   0.035035654045366904,
   0.0008099998228530136,
   0.0524518187441374,
   0.03942954735